In [1]:
import numpy as np
import pandas as pd
import os
import tqdm
import pickle
import json

import matplotlib.pyplot as plt

### Combine batched csv's

In [ ]:
scratch_dir = r'/ifs/scratch/jls106_gp/nhw2114/safegraph/'

import glob

csv_files = glob.glob(f'{scratch_dir}/*.csv')
pickle_files = glob.glob(f'{scratch_dir}/*.pickle')

In [ ]:
len(csv_files)

Bad JSON files that could not be loaded

In [ ]:
import pickle

bad_files = []
for picklepath in pickle_files:
    with open(picklepath, 'rb') as f:
        bad_files.append(pickle.load(f))

bad_files = [item for sublist in bad_files for item in sublist]
bad_files

In [ ]:
df_list = []

for filename in csv_files:
    df = pd.read_csv(filename, header=0, index_col=0)
    df_list.append(df)

df = pd.concat(df_list, axis=0)

In [ ]:
df.iloc[495:550,:]

In [ ]:
df.shape

n = df.shape[0]

18,821,974 POI-weeks

In [ ]:
count_na_cols = ['node.safegraph_geometry.wkt_area_sq_meters', 'raw_visitor_counts', 'raw_visitor_counts', 'median_dwell']

for col in count_na_cols:
    print(f"{col} has {n - sum(df.loc[:,col].isna())} rows that are not NaN")

In [ ]:
5360882 / n * 100

TODO: Check if NaNs are true zeros or visits just weren't tracked...

In [ ]:
no_nan_df = df[~df['raw_visitor_counts'].isnull()]

In [ ]:
sum(no_nan_df['raw_visit_counts'] == 0)

In [ ]:
for col in count_na_cols:
    print(f"{col} has {sum(no_nan_df.loc[:,col].isna())} rows that are NaN")

In [ ]:
no_nan_df.shape

In [ ]:
sum(no_nan_df['node.safegraph_core.naics_code'].isna())

In [ ]:
no_nan_df = no_nan_df[~no_nan_df['node.safegraph_core.naics_code'].isnull()]

In [ ]:
no_nan_df['node.safegraph_core.naics_code'] = no_nan_df['node.safegraph_core.naics_code'].astype(int).astype(str)

In [ ]:
from collections import Counter

naics_code_lengths = no_nan_df['node.safegraph_core.naics_code'].apply(len).tolist()

Counter(naics_code_lengths)

In [ ]:
len(np.unique(no_nan_df['node.safegraph_core.naics_code']))

In [ ]:
len(np.unique(no_nan_df['node.placekey']))

291 unique NAICS codes across 49,531 unique POIs

In [ ]:
no_nan_df['density'] = no_nan_df['raw_visit_counts'] / no_nan_df['node.safegraph_geometry.wkt_area_sq_meters'] 

In [ ]:
summary_df = no_nan_df[['node.safegraph_core.top_category', 'density', 'median_dwell']].groupby('node.safegraph_core.top_category').describe().unstack(1)
summary_df.to_csv('summary_stats.csv')

In [ ]:
plt.figure(figsize=(18,6))
no_nan_df[['median_dwell']].plot.hist(bins=50)

In [ ]:
np.sum(no_nan_df[['median_dwell']] < 10) / len(no_nan_df)

Not bimodal... but I think 300 minutes (5 hours) is a reasonable cutoff time

In [ ]:
plt.figure(figsize=(18,6))
no_nan_df[['density']].plot.hist(bins=50)

In [ ]:
no_nan_df.loc[no_nan_df['density'] > 300, :]

In [ ]:
removed_outlier_df = no_nan_df[no_nan_df['median_dwell'] < 500]

X = removed_outlier_df[['density','median_dwell']]

In [ ]:
X.shape

In [ ]:
X.describe()

In [ ]:
import matplotlib as mpl
mpl.rcParams['agg.path.chunksize'] = 6000000

In [ ]:
plt.figure(figsize=(18,6))
plt.plot(X.iloc[:,0], X.iloc[:,1], 'bo')
plt.xlabel('Density (weekly visit count / m^2)')
plt.ylabel('Dwell time (min)')


In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_standardized = scaler.fit_transform(X)

In [ ]:
plt.figure(figsize=(18,6))
plt.plot(X_standardized[:,0], X_standardized[:,1], 'bo')
plt.xlabel('Density')
plt.ylabel('Dwell time')

In [ ]:
import pymc3 as pm

### Kmeans

In [ ]:
from sklearn.cluster import KMeans
from tqdm import tqdm


distortions = []
for K in tqdm(range(2, 7, 2)):
    print(f"Running kmeans for K={K}")
    km = KMeans(
        n_clusters=K, init='random',
        n_init=10, max_iter=300,
        tol=1e-04, random_state=0
    )
    km.fit(X_standardized)
    distortions.append(km.inertia_)

# plot
plt.plot(range(2, 7, 2), distortions, marker='o')
plt.xlabel('Number of clusters')
plt.ylabel('Distortion')
plt.show()

In [ ]:
plt.plot(range(2, 7, 2), distortions, marker='o')
plt.xlabel('Number of clusters')
plt.ylabel('Distortion')
plt.show()

Inertia

$$
\sum_{i=0}^n \min _{\mu_j \in C}\left(\left\|x_i-\mu_j\right\|^2\right)
$$

In [ ]:
K_best = 4

km = KMeans(
    n_clusters=K_best,
    init='random',
    random_state=0
)
y_km = km.fit_predict(X_standardized)

In [ ]:
marker = ['o', 'v', '^', '<', '>', 's', '8', 'p']

color = ['lightgreen', 'lightblue', 'pink', 'yellow']


for k in range(K_best):
    plt.scatter(
        X_standardized[y_km == k, 0], X_standardized[y_km == k, 1],
        s=25, c=color[k],
        marker=marker[k], edgecolor='black',
        label=f'cluster {k+1}'
    )

# plot the centroids
plt.scatter(
    km.cluster_centers_[:, 0], km.cluster_centers_[:, 1],
    s=250, marker='*',
    c='red', edgecolor='black',
    label='centroids'
)
plt.legend(scatterpoints=1)
plt.grid()
plt.show()

In [ ]:
slice = (X_standardized[:,0] < 50)

X_zoom = X_standardized[slice, :]
y_zoom = y_km[slice]

In [ ]:
km.cluster_centers_[1:, 0]

In [ ]:
for k in range(K_best):
    plt.scatter(
        X_zoom[y_zoom == k, 0], X_zoom[y_zoom == k, 1],
        s=25, c=color[k],
        marker=marker[k], edgecolor='black',
        label=f'cluster {k+1}'
    )

# plot the centroids
plt.scatter(
    km.cluster_centers_[1:, 0], km.cluster_centers_[1:, 1],
    s=250, marker='*',
    c='red', edgecolor='black',
    label='centroids'
)
plt.legend(scatterpoints=1)
plt.grid()
plt.show()